<a href="https://colab.research.google.com/github/judem-21/Word2Vec-Model/blob/main/NSampling_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Final Word2Vec model (with Negative Sampling)

####Importing Libraries

In [1]:
import torch,torchvision
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torchvision import transforms as transforms
#from torch.torchmetrics.text.bleu import BLEUScore

In [2]:
import os
import pandas as pd
from skimage import io
import spacy
from PIL import Image
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
!python -m spacy download en
spacy_eng = spacy.load("en_core_web_sm")

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import numpy as np
import torch.nn as nn
from sklearn.decomposition import PCA
import torch
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


####Dataset Loading

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
stop_words = set(stopwords.words('english'))
class Vocabulary:
    def __init__(self, freq_threshold=3):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer(text):
      return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
           lookup=self.tokenizer(sentence)
           for word in lookup:
            if word=='\n' or word in stop_words or word in string.punctuation: continue

            if word not in frequencies:
              frequencies[word] = 1

            else:
              frequencies[word] += 1

            if frequencies[word] == self.freq_threshold:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1

In [7]:
class seq2seq_dataset:
    def __init__(self, data_file, num_samples=127584, freq_threshold=3):

        self.data = pd.read_csv(data_file).values[:num_samples]


        self.idx_sentences={}

        for idx,lines in enumerate(self.data):
            target_sentence = lines[0]
            self.idx_sentences[idx]=target_sentence

        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.idx_sentences.values())

    def __len__(self):
        return len(self.idx_sentences_source)

In [8]:
def get_dataset(
    data_file,num_samples=8000,
    freq_threshold=2):
    dataset = seq2seq_dataset(data_file, num_samples=num_samples,freq_threshold=freq_threshold)

    return dataset

In [9]:
dataset=get_dataset(data_file='/content/drive/MyDrive/Seq2SeqModel/output.csv',freq_threshold=4)

In [10]:
sentences=dataset.idx_sentences.values()

In [11]:
len(sentences)

8000

In [12]:
corpus='''Engineering is a profound and transformative discipline that fundamentally shapes human
civilization through innovative problem-solving, technological advancement, and creative design
across multiple domains. At its core, engineering represents a sophisticated synthesis of scientific
principles, mathematical precision, and practical application, enabling humanity to overcome complex
challenges and dramatically improve quality of life. From the towering skyscrapers that define modern
urban landscapes to the intricate microprocessors powering global digital infrastructure, engineers are
the architects of technological progress, continuously pushing boundaries of what is scientifically possible
and economically viable. Mechanical engineering explores the intricate dynamics of motion, energy transfer,
and mechanical systems, developing everything from microscopic medical devices to massive industrial
machinery with remarkable computational precision. Electrical engineering revolutionizes communication,
power generation, and electronic technologies, creating sophisticated networks that connect global
populations and drive digital transformation. Civil engineering constructs the physical infrastructure
supporting human societies - bridges, highways, water systems, and sustainable urban environments that
enable complex social and economic interactions. Computer engineering merges software and hardware design,
creating increasingly intelligent computational systems that reshape how humans interact with technology,
from artificial intelligence algorithms to quantum computing platforms. Chemical engineering tackles
critical global challenges by developing innovative materials, sustainable manufacturing processes, and
advanced pharmaceutical solutions that address environmental and health-related complexities.
Aerospace engineering enables humanity's exploration beyond terrestrial boundaries, designing spacecraft,
satellites, and propulsion systems that expand our understanding of the universe. Biomedical engineering
represents a cutting-edge intersection of medical science and technological innovation, developing
prosthetics, diagnostic tools, and regenerative medical technologies that dramatically improve human
health outcomes. Environmental engineering addresses urgent global sustainability challenges, designing
solutions for renewable energy, waste management, and ecological conservation. Each engineering discipline
contributes uniquely to human progress, representing a dynamic, intellectually rigorous profession that
demands continuous learning, creativity, and a profound commitment to solving increasingly complex global
challenges through systematic, innovative approaches. Engineering represents an extraordinary intellectual and
technological frontier that transcends traditional disciplinary boundaries, embodying humanity's most ambitious aspirations
for scientific innovation, societal transformation, and global problem-solving through sophisticated computational methodologies,
advanced design principles, and interdisciplinary collaboration. This monumentally complex profession encompasses an intricate, dynamically
evolving network of specialized domains, each contributing uniquely to human progress through rigorous scientific investigation, cutting-edge
technological development, and systematic approaches to addressing increasingly complex global challenges across multiple dimensions of human
experience. Mechanical engineering emerges as a foundational discipline that meticulously explores the intricate interactions between complex
physical systems, materials science, energy transfer mechanisms, and dynamic computational modeling, developing revolutionary technologies
ranging from microscopic medical nanomachines to massive industrial infrastructure, aerospace propulsion systems, and advanced robotic platforms
that dramatically expand human technological capabilities and scientific understanding. Electrical engineering fundamentally revolutionizes global
communication, computational infrastructure, and technological ecosystems by creating sophisticated networks, semiconductor technologies, quantum
computing architectures, and advanced electronic systems that interconnect billions of individuals, transform information exchange paradigms,
and enable unprecedented levels of computational intelligence and technological integration across diverse global contexts. Computer engineering
represents the quintessential frontier of technological innovation, seamlessly merging hardware and software design principles to create increasingly
           intelligent computational platforms, artificial intelligence algorithms, machine learning ecosystems, and transformative digital technologies that reshape human
           interaction, decision-making processes, and technological engagement through increasingly complex and adaptive computational frameworks. Civil engineering
           constructs the fundamental physical infrastructure supporting human civilization, designing resilient, sustainable, and technologically advanced urban environments,
           transportation networks, water management systems, and architectural solutions that address complex social, economic, and environmental challenges while simultaneously
           promoting human connectivity, resource efficiency, and ecological sustainability. Chemical engineering tackles critical global challenges by developing innovative
           materials, sustainable manufacturing processes, advanced pharmaceutical solutions, and transformative biotechnological platforms that address environmental degradation,
           energy efficiency, human health complexities, and the intricate interactions between technological systems and natural ecosystems. Aerospace engineering enables humanity's
           most audacious technological dreams, designing sophisticated spacecraft, satellite systems, propulsion technologies, and exploration platforms that expand human understanding
           of the universe, push the boundaries of scientific exploration, and create revolutionary technologies for interplanetary research and technological innovation. Biomedical
           engineering emerges as a groundbreaking interdisciplinary domain, combining medical science, computational technologies, and advanced engineering principles to develop
           regenerative medical solutions, personalized healthcare platforms, advanced diagnostic technologies, prosthetic systems, and transformative medical interventions that
           dramatically improve human health outcomes, extend medical capabilities, and create unprecedented opportunities for technological intervention in human physiological systems.
           Environmental engineering addresses urgent global sustainability challenges by creating innovative solutions for renewable energy generation, ecological conservation, climate
           change mitigation, waste management, and sustainable technological development that are increasingly critical to human survival, planetary health, and long-term ecological
           equilibrium. Each engineering discipline contributes a unique, sophisticated perspective to the grand narrative of human technological progress, representing a dynamic,
           intellectually rigorous profession that demands continuous learning, extraordinary creativity, interdisciplinary collaboration, and an unwavering commitment to solving
           increasingly complex global challenges through systematic, innovative, and transformative approaches that push the boundaries of human knowledge and technological capability.'''

'''sentences=corpus.split('.')
tokenised_words=word_tokenize(corpus)
stop_words = set(stopwords.words('english'))'''

"sentences=corpus.split('.')\ntokenised_words=word_tokenize(corpus)\nstop_words = set(stopwords.words('english'))"

####Data PreProcess

In [13]:
def data_extraction(sentences,string2idx,window_size=2):
  data=[]
  for sentence in sentences:
    words=[tok.text.lower() for tok in spacy_eng.tokenizer(sentence)]
    for i in range(len(words)):
      token=words[i].lower()
      if token in string.punctuation or token in stop_words or token not in string2idx: continue
      for j in range(max(0,i-window_size),min(len(words),i+window_size+1)):
        if i!=j:
          if words[j].lower() not in stop_words and words[j] not in string.punctuation and words[j].lower() in string2idx:data.append([string2idx[words[i].lower()],string2idx[words[j].lower()]])
  return data

In [14]:
string2idx=dataset.vocab.stoi
data=data_extraction(sentences,string2idx,window_size=5)
data=torch.tensor(data)

In [15]:
data.shape

torch.Size([151406, 2])

####Model

In [16]:
class SkipGram_model(nn.Module):
  def __init__(self,vocab_size,embedding_dim):
    super(SkipGram_model,self).__init__()
    self.embeddings_centre=nn.Embedding(vocab_size,embedding_dim)
    self.embeddings_context=nn.Embedding(vocab_size,embedding_dim)

  def forward(self,centre,context):
    centre_embed=self.embeddings_centre(centre)
    context_embed=self.embeddings_context(context)
    scores=torch.cosine_similarity(centre_embed,context_embed,dim=1)
    return scores

In [33]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=SkipGram_model(vocab_size=len(string2idx),embedding_dim=32).to(device)
loss=nn.BCEWithLogitsLoss()

####Training

In [32]:
def train(model,centre,context,vocab_size,loss_function,optimizer,batch_size,num_epochs=1000,lr=0.01,negative_sampling_ratio=5,device='cuda'):
  num_batches=centre.shape[0]//batch_size
  epoch_loss=[]
  for epoch in range(num_epochs):
    batch_loss=[]
    for batch in range(num_batches):
      batch_centre=centre[batch*batch_size:(batch+1)*batch_size].to(device)
      batch_context=context[batch*batch_size:(batch+1)*batch_size].to(device)

      #positive_samples
      pos_label=torch.ones(batch_size).to(device)
      pos_out=model(batch_centre,batch_context)
      pos_loss=loss_function(pos_out,pos_label)

      #negative_samples
      counter=0
      neg_loss=0
      while counter<negative_sampling_ratio:
        neg_context=torch.randint(low=0,high=vocab_size,size=(batch_size,)).to(device)
        if (batch_context==neg_context).all() or (batch_centre==neg_context).any(): continue
        neg_label=torch.zeros(batch_size).to(device)
        neg_out=model(batch_centre,neg_context)
        neg_loss+=loss_function(neg_out,neg_label)
        counter+=1
      total_loss=pos_loss+(neg_loss/negative_sampling_ratio)
      batch_loss.append(total_loss.item())
      if (epoch==0 and batch==0) or (((epoch+1)%(num_epochs//5))==0 and ((batch+1)%1000)==0):
        print(f'Epoch {epoch+1}-Batch {batch+1}:- Loss={total_loss.item()}')
      optimizer.zero_grad()
      total_loss.backward()
      optimizer.step()
    total_loss=np.mean(batch_loss)
    epoch_loss.append(total_loss)
  return model,epoch_loss

In [ ]:
epochs=10000;lr=0.001
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
batch_size=64
total_samples=batch_size*(len(data)//batch_size)
centre=data[:total_samples,0]
context=data[:total_samples,1].to(device)
model,epoch_loss=train(model,centre,context,len(string2idx),loss,optimizer,batch_size,num_epochs=epochs,lr=lr,negative_sampling_ratio=5,device=device)

Epoch 1-Batch 1:- Loss=1.4023714065551758


####Testing

In [ ]:
word_embeddings=model.embeddings.weight.detach().cpu().numpy()
embedding_choices=np.random.choice(np.arange(len(string2idx)),size=100,replace=False)
print(f'Word embeddings shape: {word_embeddings[embedding_choices].shape}')
pca=PCA(n_components=3).fit_transform(word_embeddings[embedding_choices])
print(f'pca shape: {pca.shape}')
fig=plt.figure(figsize=(20,20))
ax=fig.add_subplot(111,projection='3d')
for idx,word_idx in enumerate(embedding_choices):
    #print(word,idx)
    x,y,z=pca[idx]
    ax.scatter(x,y,z,color='blue')
    ax.text(x,y,z,dataset.vocab.itos[word_idx])
plt.show()